## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Morro Bay,US,35.3658,-120.8499,68.14,88,0,0.67,clear sky
1,1,Korla,CN,41.7597,86.1469,83.55,17,0,5.21,clear sky
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,42.80,93,40,5.75,scattered clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,48.31,66,75,12.66,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,53.22,62,89,4.61,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                251
City                   251
Country                249
Lat                    251
Lng                    251
Max Temp               251
Humidity               251
Cloudiness             251
Wind Speed             251
Current Description    251
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Korla,CN,83.55,clear sky,41.7597,86.1469,
5,Bac Lieu,VN,79.59,light rain,9.2850,105.7244,
11,Bydgoszcz,PL,77.56,clear sky,53.1235,18.0076,
12,Cayenne,GF,87.84,broken clouds,4.9333,-52.3333,
16,Madimba,TZ,79.92,broken clouds,-10.4000,40.3333,
17,Victoria,HK,88.32,light rain,22.2855,114.1577,
18,Mayumba,GA,79.09,clear sky,-3.4320,10.6554,
21,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,
22,Faanui,PF,78.58,light rain,-16.4833,-151.7500,
23,Samana,DO,84.16,scattered clouds,19.2056,-69.3369,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Korla,CN,83.55,clear sky,41.7597,86.1469,Gulisitan Hotel
5,Bac Lieu,VN,79.59,light rain,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu
11,Bydgoszcz,PL,77.56,clear sky,53.1235,18.0076,Słoneczny młyn
12,Cayenne,GF,87.84,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
17,Victoria,HK,88.32,light rain,22.2855,114.1577,Mini Hotel Central
18,Mayumba,GA,79.09,clear sky,-3.4320,10.6554,LIKWALE LODGE
21,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,Lindsey Hôtel
22,Faanui,PF,78.58,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
23,Samana,DO,84.16,scattered clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
26,Kapaa,US,79.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))